迭代协议：用\_\_iter__方法返回了一个实现了\_\_next__方法的迭代器对象。

迭代器使用分离设计：对目标对象而言，迭代器是一种与自身逻辑无关的用户接口，组合比内联更合适；迭代分多次完成，需要保存进度，而且可能有重复迭代、同时多个迭代的情形。

In [2]:
class Data:
    def __init__(self, n):
        self.data = list(range(n))
        
    def __iter__(self):
        return DataIter(self.data)

In [4]:
class DataIter:
    def __init__(self, data):
        self.data = data
        self.index = 0
    def __next__(self):
        if not self.data or self.index >= len(self.data):
            raise StopIteration
        
        d = self.data[self.index]
        self.index += 1
        return d

In [10]:
d = Data(2)
x = d.__iter__()

In [11]:
x.__next__()

0

In [12]:
next(x)

1

In [13]:
next(x)

StopIteration: 

In [14]:
for i in Data(3):
    print(i)

0
1
2


In [15]:
class Data2:
    def __init__(self, n):
        self.data = list(range(n))
    
    def __iter__(self):
        return iter(self.data)   # 辅助函数，简便的方式

In [16]:
y = Data2(2).__iter__()

In [17]:
y.__next__()

0

In [18]:
y.__next__()

1

In [19]:
next(y)

StopIteration: 

In [20]:
x = iter([1,2,3])
while True:
    print(next(x))

1
2
3


StopIteration: 

尽管列表、字典等容器类型实现了迭代器协议，但本质上其和迭代器属于不同层面，迭代器不仅是一种数据读取方法，更多的是一种设计模式。

容器的核心是存储，围绕数据提供操作方法，是与用户逻辑无关的开放类型。而迭代器的重点是逻辑控制，调用方发出请求，随后决策由迭代器决定，数据内敛，抽象和实现分离。

生成器是迭代器的进化版本，其用函数和表达式替代接口方法，提供了更多控制能力用于复杂的设计。

In [21]:
def test():
    for i in range(10):
        yield i+100
        if i > 1:
            return 
x = test()

In [22]:
next(x)

100

In [23]:
next(x)

101

In [24]:
next(x)

102

In [25]:
next(x)

StopIteration: 

生成器的执行过程：首先编译器为生成器函数添加标记。对此类函数，解释器不直接执行，而是将栈帧和代码做为参数，创建生成器实例。当执行到yield指令时，解释器设置好返回值并保存线程状态，并挂起当前函数流程。只有再次调用\_\_next__方法时，才能恢复状态，继续执行，直到函数结束。

In [26]:
def test(n):
    print('gen start')
    for i in range(n):
        print('gen yield ', i)
        yield i
        print('gen.resume')

In [27]:
test.__code__.co_flags

99

In [33]:
import inspect
inspect.isgeneratorfunction(test)

True

In [35]:
x = test(2)
x.gi_frame.f_locals

{'n': 2}

In [36]:
x.__next__  # 实现迭代器协议方法

<method-wrapper '__next__' of generator object at 0x10c984200>

In [38]:
x.__next__()

gen start
gen yield  0


0

In [39]:
x.__next__()

gen.resume
gen yield  1


1

In [40]:
x.__next__()

gen.resume


StopIteration: 

生成器的另一个特征就是提供双向通信能力。

In [41]:
def test():
    while True:
        v = yield 200
        print('resume ',v)

In [46]:
x = test()
x.send(None)  # 必须先发送 None 启动生成器，或先用next()启动

200

In [48]:
x.send('dsdsd') # 可发送任意数据

resume  dsdsd


200

In [49]:
x.close() # 终止生成器

对于生成器函数而言，挂起点是一个安全的位置，相关状态临时冻结，只要将要发送的数据或其他状态标记放在栈帧指定位置，由解释器决定如何处理。  
close方法在生成器内部引发GeneratorExit异常，通知解释器结束执行，该异常不可被捕获

In [52]:
class ExitException(Exception):pass
class ResetException(Exception):pass

def test():
    while True:
        try:
            v = yield
            print('recive', v)
        except ResetException:
            print('reset logic')
        except ExitException:
            print('exit')
            return 
x=test()

In [53]:
next(x) # 启动生成器

In [54]:
next(x)

recive None


In [55]:
x.throw(ResetException) # 发出重置信号后可继续发送数据

reset logic


In [56]:
x.send('ss')

recive ss


In [57]:
x.throw(ExitException)

exit


StopIteration: 

In [60]:
# 回调模式
import time
import threading
def target(request, callback):
    request()  # 调用请求函数
    time.sleep(2) # 模拟阻塞情况
    callback('sth')   #  调用回调函数
    
def service(request, callback):
    threading.Thread(target=target, args=(request, callback)).start()
    
def request():
    print('request start')

def callback(x):
    print(x)
service(request, callback)

request start
sth


In [65]:
# 消除回调的生成器模式, 消除碎片化
def request():
    print('request start1')
    x = yield
    print(x)
    
def target(fn):
    try:
        g = fn()
        g.send(None)
        time.sleep(2)
        g.send('sth1')
    except StopIteration:
        pass

def service(fn):
    threading.Thread(target=target, args=(fn,)).start()
service(request)

request start1
sth1


协程以协作调度的方式，在单个线程上切换执行并发任务，可将IO阻塞时间用来执行更多任务，也被称作用户线程。

In [67]:
def sched(*tasks):
    tasks = list(map(lambda t: t(), tasks))  # 调用所有任务函数，一个生成器的列表
    while tasks:
        try:
            t = tasks.pop(0) # 列表头部弹出任务
            t.send(None)  # 开始执行
            tasks.append(t) # 如果任务没有结束，则放回列表尾部
        except StopIteration: # 任务结束，丢弃
            pass

In [68]:
from functools import partial

def task(id,n,m):
    for i in range(n,m):
        print(f"{id}:{i}")
        yield
t1 = partial(task, 1, 10, 33)
t2 = partial(task, 2, 30, 35)
sched(t1,t2)

1:10
2:30
1:11
2:31
1:12
2:32
1:13
2:33
1:14
2:34
1:15
1:16
1:17
1:18
1:19
1:20
1:21
1:22
1:23
1:24
1:25
1:26
1:27
1:28
1:29
1:30
1:31
1:32


函数式编程三大特征：第一类型函数、不可变数据、尾递归优化。

命令式  
x = a+b  
y = x+2  
函数式  
mul(add(a,b),2)  
add(a,b).mul(2)

迭代：以迭代器的方式，用指定函数处理数据源。相比推导式，它能从多个数据源中平行接收参数，直至最短数据源迭代完成。

In [70]:
x = map(lambda a,b: (a,b), [1,2,3], 'abcd')

In [71]:
list(x)

[(1, 'a'), (2, 'b'), (3, 'c')]

聚合：平行从多个数据源接收参数，聚合成元组，直到最短数据源结束。最长数据源可以用itertools.zip_longest()

In [72]:
x = zip((1,2,3),'abcd',[1.1,2.2])

In [73]:
list(x)

[(1, 'a', 1.1), (2, 'b', 2.2)]

累积：迭代数据源，将结果带入下次计算。

In [75]:
import functools
def calc(ret, x):
    print(f'ret={ret}, x={x}')
    return ret+x

In [77]:
functools.reduce(calc, [1,2,3,4,5,6])

ret=1, x=2
ret=3, x=3
ret=6, x=4
ret=10, x=5
ret=15, x=6


21

In [79]:
functools.reduce(calc, [1,2,3,4,5,6], 100)  # 添加初始化值

ret=100, x=1
ret=101, x=2
ret=103, x=3
ret=106, x=4
ret=110, x=5
ret=115, x=6


121

过滤：使用指定函数对数据进行过滤

In [80]:
x = filter(lambda n: n%2==0, range(10))
list(x)

[0, 2, 4, 6, 8]

In [81]:
list(filter(None, [0, 1, [], '', 1.1]))

[1, 1.1]

判断：一系列数据中，全部或某个为真值

In [83]:
all([1,'a',''])

False

In [84]:
any([1,'a',''])

True

In [1]:
# 迭代器切片
def count(n):
    while True:
        yield n
        n += 1
c = count(0)
c[10:20]

TypeError: 'generator' object is not subscriptable

In [2]:
import itertools
for x in itertools.islice(c, 10, 20):
    print(x)

10
11
12
13
14
15
16
17
18
19


In [3]:
# 展开嵌套的序列
from collections import Iterable

def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x

items = [1, 2, [3, 4, [5, 6], 7], 8]
# Produces 1 2 3 4 5 6 7 8
for x in flatten(items):
    print(x)

1
2
3
4
5
6
7
8
